In [ ]:
pip install requests newspaper3k lxml_html_clean bs4 fake_useragent python-docx


In [ ]:
# The guardian
# Premium times
# The Nation


In [1]:
import os

# Get all .txt files in the working directory
for filename in os.listdir("."):
    if filename.endswith(".txt"):
        # Read the file
        with open(filename, "r", encoding="utf-8") as f:
            content = f.read()
        
        # Replace literal "\n" with commas and split into unique strings
        modified_content = content.split("\\n")   # splits on the literal backslash-n
        modified_content = set(modified_content)  # remove duplicates

        # Join back into a single string (comma-separated)
        new_text = ",".join(modified_content)

        # Overwrite the file with the new content
        with open(filename, "w", encoding="utf-8") as f:
            f.write(new_text)

        print(f"Processed: {filename}")


Processed: done_guardianng_rape_articles.txt
Processed: done_guardianng_rape_articles_revelance.txt
Processed: done_guardianng_sexual_assualt_articles.txt
Processed: done_guardianng_sexual_assualt_articles_relevance.txt
Processed: done_premium_times_rape_articles.txt
Processed: done_premium_times_sexual_assault_articles.txt
Processed: done_premiumtimesngcom_rape_articles_revelance.txt
Processed: done_premiumtimesngcom_sexual_assault_articles_revelance.txt
Processed: done_thenationonlineng.net_rape_articles.txt
Processed: done_thenationonlinengnet_rape_articles_revelance.txt
Processed: done_thenationonlinengnet_sexual_assault_articles.txt
Processed: done_thenationonlinengnet_sexual_assault_articles_revelance.txt
Processed: output.txt
Processed: done_punch_rape_all_articles.txt


In [8]:
import requests
from newspaper import Article, Config

def full_content(url):
    user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'
    config = Config()
    config.request_timeout = 10
    config.browser_user_agent = user_agent
    page = Article(url, config = config)
    

    try:
        page.download()
        page.parse()
        return page.text, page.authors, page.publish_date
    except Exception as e:
        print(f"Error retrieving content from {url}: {e}")
        return 'couldnt retrieve'

full_content('https://www.vanguardngr.com/2025/09/suarez-banned-six-matches-for-leagues-cup-spitting-furore/')

Error retrieving content from https://www.vanguardngr.com/2025/09/suarez-banned-six-matches-for-leagues-cup-spitting-furore/: Article `download()` failed with 403 Client Error: Forbidden for url: https://www.vanguardngr.com/2025/09/suarez-banned-six-matches-for-leagues-cup-spitting-furore/ on URL https://www.vanguardngr.com/2025/09/suarez-banned-six-matches-for-leagues-cup-spitting-furore/


'couldnt retrieve'

In [ ]:
# Read file, replace \n with comma, make each line a string
with open('/workspace/testing/punch_rape_all_articles.txt', 'r') as file:
    content = file.read()

# Replace newlines with commas and split into strings
modified_content = content.split('\\n')

modified_content = (set(modified_content))
modified_content


{'',
 'https://punchng.com/',
 'https://punchng.com/10-year-old-rape-victim-bled-three-days-after-incident-witness/',
 'https://punchng.com/11200-nigerian-women-children-raped-in-2020-un/',
 'https://punchng.com/14-year-old-rape-victim-contracts-hiv-culprit-bags-life-jail/',
 'https://punchng.com/20-ways-to-protect-yourself-from-rapists/',
 'https://punchng.com/20-year-old-lady-accuses-cop-of-sexual-harassment-ogun-police-begin-probe/',
 'https://punchng.com/26-year-old-man-defiles-four-year-old-pupil-in-oyo/',
 'https://punchng.com/39-lecturers-indicted-for-sexual-harassment-in-five-years-report/',
 'https://punchng.com/52-year-old-abuja-bishop-bags-20-year-jail-for-raping-minor/',
 'https://punchng.com/60-year-old-man-arrested-for-raping-lagos-lady/',
 'https://punchng.com/60-year-old-man-faces-court-for-alleged-sexual-assault-in-lagos/',
 'https://punchng.com/60-year-old-man-faces-court-for-alleged-sexual-assault-in-lagos/?utm_source=auto-read-also&utm_medium=web&amp',
 'https://pun

In [4]:
import requests
from newspaper import Article, Config
from docx import Document
import re
from datetime import datetime
import os

def clean_filename(filename):
    """Clean filename by removing invalid characters"""
    # Remove or replace invalid characters for filenames
    filename = re.sub(r'[<>:"/\\|?*]', '', filename)
    filename = re.sub(r'[^\w\s-]', '', filename)
    filename = re.sub(r'[-\s]+', '-', filename)
    return filename.strip('-')

def get_article_and_save_doc(url):
    """
    Get article content, title, date and save as Word doc
    Filename format: "Title - Date.docx"
    """
    user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'
    config = Config()
    config.browser_user_agent = user_agent
    page = Article(url, config=config)

    try:
        print(f"Downloading article from: {url}")
        page.download()
        page.parse()
        
        # Get article details
        title = page.title if page.title else "Untitled Article"
        content = page.text if page.text else "No content retrieved"
        publish_date = page.publish_date
        authors = page.authors if page.authors else ["Unknown Author"]
        
        # Format date for filename
        if publish_date:
            date_str = publish_date.strftime("%Y-%m-%d")
        else:
            date_str = datetime.now().strftime("%Y-%m-%d")
        
        # Create filename: "Title - Date.docx"
        filename = f"{clean_filename(title)} - {date_str}.docx"
        
        # Limit filename length to avoid issues
        if len(filename) > 200:
            short_title = clean_filename(title)[:150]
            filename = f"{short_title} - {date_str}.docx"
        
        # Create Word document
        doc = Document()
        
        # Add title as heading
        doc.add_heading(title, 0)
        
        # Add metadata
        doc.add_paragraph(f"Date: {date_str}")
        doc.add_paragraph(f"Source: {url}")
        doc.add_paragraph("")  # Empty line
        
        # Add content
        # Split content into paragraphs and add each one
        paragraphs = content.split('\\n\\n')
        for paragraph in paragraphs:
            if paragraph.strip():
                doc.add_paragraph(paragraph.strip())
        
        # Save document
        folder_path = "/workspace/testing/the_punch_rape_doc_files" # Example for Windows
        # Or: folder_path = "/home/youruser/MyWordDocuments" # Example for Linux/macOS
        

        # Ensure the folder exists (create it if it doesn't)
        os.makedirs(folder_path, exist_ok=True)

        # Construct the full path
        full_path = os.path.join(folder_path, filename)

        # Save the document to the specified path
        doc.save(full_path)
        
        
        print(f"✅ Article saved as: {filename}")
        print(f"Title: {title}")
        print(f"Date: {date_str}")
        print(f"Authors: {', '.join(authors)}")
        print(f"Content length: {len(content)} characters")
        
        return {
            'title': title,
            'date': publish_date,
            'authors': authors,
            'content': content,
            'filename': filename,
            'success': True
        }
        
    except Exception as e:
        print(f"❌ Error retrieving content from {url}: {e}")
        return {
            'error': str(e),
            'url': url,
            'success': False
        }

def process_multiple_urls(urls):
    """
    Process multiple URLs and save each as a separate doc file
    """
    results = []
    
    for i, url in enumerate(urls, 1):
        print(f"\\n--- Processing article {i}/{len(urls)} ---")
        result = get_article_and_save_doc(url)
        results.append(result)
        
        if result['success']:
            print(f"Success: {result['filename']}")
        else:
            print(f"Failed: {url}")
    
    # Summary
    successful = len([r for r in results if r['success']])
    print(f"\\n=== SUMMARY ===")
    print(f"Total articles processed: {len(urls)}")
    print(f"Successful: {successful}")
    print(f"Failed: {len(urls) - successful}")
    
    return results

# Example usage for single article
if __name__ == "__main__":
    # Single article
    # url = 'https://thenationonlineng.net/bbnaija-s10-19-housemates-face-possible-eviction/'
    # result = get_article_and_save_doc(url)
    
    # Multiple articles example (uncomment to use)
    # urls = [
    #     'https://thenationonlineng.net/bbnaija-s10-19-housemates-face-possible-eviction/',
    #     'https://example.com/article2',
    #     'https://example.com/article3'
    # ]
    results = process_multiple_urls(modified_content)

\n--- Processing article 1/286 ---


✅ Article saved as: Two-men-bag-life-jail-for-gang-raping-Ekiti-minor - 2025-03-05.docx
Title: Two men bag life jail for gang-raping Ekiti minor
Date: 2025-03-05
Authors: Abiodun Nejo
Content length: 1602 characters
Success: Two-men-bag-life-jail-for-gang-raping-Ekiti-minor - 2025-03-05.docx
\n--- Processing article 2/286 ---
✅ Article saved as: Lagos-medical-director-appeals-rape-conviction - 2023-12-06.docx
Title: Lagos medical director appeals rape conviction
Date: 2023-12-06
Authors: Solomon Odeniyi
Content length: 2455 characters
Success: Lagos-medical-director-appeals-rape-conviction - 2023-12-06.docx
\n--- Processing article 3/286 ---
❌ Error retrieving content from : Article `download()` failed with No connection adapters were found for '://' on URL ://
Failed: 
\n--- Processing article 4/286 ---
✅ Article saved as: Many-police-officers-are-hostile-to-female-rape-victims-Consultant-psychologist - 2020-06-15.docx
Title: Many police officers are hostile to female rape victims –Co

In [ ]:
pip install pandas